In [ ]:
# Cell 1: Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import torch
import torchvision
from torchvision import models, transforms
from torchvision.models.detection import maskrcnn_resnet50_fpn
from PIL import Image
import os

cuda_available = torch.cuda.is_available()

print(f"CUDA Available: {cuda_available}")

if cuda_available:
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available. PyTorch will use the CPU.")

In [ ]:
import torchvision

model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights=torchvision.models.detection.MaskRCNN_ResNet50_FPN_Weights.DEFAULT)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()


In [ ]:
# Cell 3: Define the COCO labels
coco_labels = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

vehicle_labels = ['car', 'motorcycle', 'bus', 'truck']

In [ ]:
# Cell 4: Define the transformation
transform = transforms.Compose([
    transforms.ToTensor()
])

In [ ]:
# Cell 5: Detect objects in an image
def detect_objects(image):
    image_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image_tensor)

    boxes = outputs[0]['boxes'].cpu().numpy()
    labels = outputs[0]['labels'].cpu().numpy()
    scores = outputs[0]['scores'].cpu().numpy()
    masks = outputs[0]['masks'].cpu().numpy()

    detected_objects = []

    for box, label, score, mask in zip(boxes, labels, scores, masks):
        if score >= 0.5:
            detected_objects.append((box, coco_labels[label], score, mask))

    return detected_objects

In [ ]:
def adjust_coordinates(box, original_size, preprocessed_size):
    original_width, original_height = original_size
    preprocessed_width, preprocessed_height = preprocessed_size

    scale_width = original_width / preprocessed_width
    scale_height = original_height / preprocessed_height

    xmin, ymin, xmax, ymax = box
    xmin = int(xmin * scale_width)
    xmax = int(xmax * scale_width)
    ymin = int(ymin * scale_height)
    ymax = int(ymax * scale_height)

    return xmin, ymin, xmax, ymax

def visualize_detections(image_path, detected_objects, preprocessed_image, confidence_threshold=0.7):
    image = cv2.imread(image_path)
    image_with_detections = image.copy()

    original_size = (image.shape[1], image.shape[0])  # (width, height)
    preprocessed_size = (preprocessed_image.shape[1], preprocessed_image.shape[0])  # (width, height)

    for box, label, score in detected_objects:
        if score >= confidence_threshold:
            adjusted_box = adjust_coordinates(box, original_size, preprocessed_size)
            xmin, ymin, xmax, ymax = adjusted_box

            cv2.rectangle(image_with_detections, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
            cv2.putText(image_with_detections, f"{label}: {score:.2f}", (xmin, ymin - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    return image_with_detections


In [ ]:
# Cell 7: Preprocess the image
def preprocess_image(image_path, target_size=(800, 800)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    h, w, _ = image.shape
    scale = min(target_size[0] / w, target_size[1] / h)
    resized_image = cv2.resize(image, None, fx=scale, fy=scale)
    
    padded_image = np.zeros((target_size[0], target_size[1], 3), dtype=np.uint8)
    h, w, _ = resized_image.shape
    padded_image[:h, :w, :] = resized_image
    
    return padded_image

In [ ]:
# Cell 8: Test object detection with preprocessing
import glob

dataset_path = "dataset/raw-images"
rgb_image_paths = []

for subfolder in os.listdir(dataset_path):
    if "rgb" in subfolder.lower():
        subfolder_path = os.path.join(dataset_path, subfolder)
        if os.path.isdir(subfolder_path):
            rgb_image_paths.extend(glob.glob(os.path.join(subfolder_path, "*.jpg")))
            rgb_image_paths.extend(glob.glob(os.path.join(subfolder_path, "*.png")))

np.random.shuffle(rgb_image_paths)

num_test_images = 10
test_image_paths = rgb_image_paths[:num_test_images]

# Perform object detection on the test images with preprocessing
for image_path in test_image_paths:
    original_image = cv2.imread(image_path)
    preprocessed_image = preprocess_image(image_path)
    print("Preprocessed Image Shape:", preprocessed_image.shape)
    
    pil_image = Image.fromarray(preprocessed_image)
    
    detected_objects = detect_objects(pil_image)
    print("Detected Objects:")
    for obj in detected_objects:
        print(obj)
    
    scale_x = original_image.shape[1] / preprocessed_image.shape[1]
    scale_y = original_image.shape[0] / preprocessed_image.shape[0]
    
    image_with_detections = original_image.copy()
    
    for box, label, score, _ in detected_objects:
        if score >= 0.5:
            xmin, ymin, xmax, ymax = box
            xmin_scaled = int(xmin * scale_x)
            ymin_scaled = int(ymin * scale_y)
            xmax_scaled = int(xmax * scale_x)
            ymax_scaled = int(ymax * scale_y)
    
            cv2.rectangle(image_with_detections, (xmin_scaled, ymin_scaled), (xmax_scaled, ymax_scaled), (0, 255, 0), 2)
            cv2.putText(image_with_detections, f"{label}: {score:.2f}", (xmin_scaled, ymin_scaled - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    
    print("Image with Detections Shape:", image_with_detections.shape)
    
    cv2.imshow("Image with Detections", image_with_detections)
    cv2.waitKey(0)
    cv2.destroyAllWindows()